In [1]:
import sys
sys.path.append("C:\Program Files\Tinysoft\Analyse.NET")
sys.path.append(r"C:\Users\xudong\Documents\github\coresearch\funcs")
sys.path.append(r"C:\Users\xudong\Documents\guanzhao\dataserver")
import pymongo
import numpy as np
import pickle
import time
import datetime
import copy
import pandas as pd
from operator import itemgetter
import TSLPy3 as tsl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import talib
import importlib
import rschLib
from pylab import mpl
from library.tags import tags
np.set_printoptions(formatter={'float_kind': "{:.6f}".format})

db = rschLib.db_quanLiang()
dbt = rschLib.db_tinySoftData()
dtes, tkrs, name, open_mtx, high_mtx, low_mtx, close_mtx, belong, shenwan1, shenwan2, shenwan3, vol_mtx, amount_mtx = rschLib.loadDailyBarMtx("d:\\pklWeeklyUpdate\\")


init success


In [2]:
def logTime(s):
    print(s, datetime.datetime.now())

def generateTagAnalysis(tagName):
    print(tagName)
    tname = rschLib.tagMapper(tagName)['file']
    strategy_name = tname
    db.strategyBackTest.remove({'strategy_name':strategy_name})
    db.strategyBackTestTrades.remove({'strategy_name':strategy_name})
    db.strategyBackTest.update_one({'strategy_name': strategy_name}, {'$set':{
        'strategy_name':strategy_name
        }},upsert=True)
    startDate = 20180101
    with open("d:\\pklWeeklyUpdate\\" + tname + ".pkl", 'rb+') as f:
        tagInfo = pickle.load(f)
    tag_mtx = tagInfo['tag_mtx']
    off_start = rschLib.tagMapper(tagName)['off_start']
    rschLib.saveOffStart(strategy_name, off_start)
    tagNotNew = np.zeros(close_mtx.shape)
    for i in range(close_mtx.shape[0]):
        j = np.nonzero(close_mtx[i,:]>0)[0][0]
        v = np.min((close_mtx.shape[1], j+60))
        tagNotNew[i, v:]=1
    tag_mtx = (tag_mtx==1) & (tagNotNew==1) # 去掉新股票
    
    dtesUsed = dtes[dtes>=startDate]
    tag_mtxUsed = tag_mtx[:, dtes>=startDate]
    for (i, x) in enumerate(dtesUsed):
        q = name[tag_mtxUsed[:,i]==1]
        tk = tkrs[tag_mtxUsed[:,i]==1]
        query = []
        for i, y in enumerate(q):
            query.append({
                'strategy_name':strategy_name,
                'name':y,
                'ticker':tk[i],
                'dateIn':int(x)
            })
        if len(query)>0:
            db.strategyBackTestTrades.insert_many(query)
    rschLib.updateStrategyGeneratingStatus(strategy_name, '生成进度:10%。初始化标签。'+str(datetime.datetime.now()),10)
    rschLib.updateStrategyPrivacy(strategy_name, 'public')
    

In [3]:
def analyzeStrategy(strategy_name, offStart, dtes, name, tkrs,maxD, inTime, otTime):
    timeAsFloat, timeLabels, maxM, dayOff, dayTimeAsFloat = rschLib.getTimeLabels(maxD)
    # get trades
    R = open_mtx[:, 1:]/close_mtx[:,:-1]-1  #使用收盘到开盘的回报率来修正分红和拆股
    R = np.hstack((np.zeros((R.shape[0],1)), R))
    logTime('getTradesFast')
    tradesUsed, r_withnan = rschLib.getTradesFast(strategy_name, name, tkrs, dtes, maxD, dayTimeAsFloat, R)
    logTime('getTradesFast done')
    # get trade samples by good/bad trades
    tradeArea=[inTime,otTime]
    r = r_withnan.copy()
    r[np.isfinite(r)==False]=0
    # draw price change
    idxTradable = np.isfinite(r_withnan[:,tradeArea[0]])
    logTime('getTradesAnalysisSampleGroups')
    result = rschLib.getTradeAnalysisSampleGroups(r, idxTradable, tradeArea)
    logTime('getTradesAnalysisSampleGroups done')
    logTime('drawPriceChange')
    rschLib.drawPriceChange(r[idxTradable,:], strategy_name, timeLabels=timeLabels, tp=tradeArea, showGraph='no')
    rschLib.drawPriceChange(result['rGood10'], strategy_name, timeLabels=timeLabels, title='盈利前10%交易', tp=tradeArea, showGraph='no')
    rschLib.drawPriceChange(result['rGood30'], strategy_name, timeLabels=timeLabels, title='盈利前30%交易', tp=tradeArea, showGraph='no')
    rschLib.drawPriceChange(result['rBad10'], strategy_name, timeLabels=timeLabels, title='亏损前10%交易',  tp=tradeArea, showGraph='no')
    rschLib.drawPriceChange(result['rBad30'], strategy_name, timeLabels=timeLabels, title='亏损前30%交易',  tp=tradeArea, showGraph='no')
    rschLib.drawPriceChange(result['rDieting'], strategy_name, timeLabels=timeLabels, title='第一天跌停', tp=tradeArea, showGraph='no')
    logTime('drawPriceChange done')
    # analyze tags
    #rschLib.analyzeTradeTags(tradesUsed, result['rGood10'], result['idxGood10'], '盈利前10%交易',strategy_name, dtes, tkrs, offStart,"d:\\pklWeeklyUpdate\\")
    #rschLib.analyzeTradeTags(tradesUsed, result['rGood30'], result['idxGood30'], '盈利前30%交易',strategy_name, dtes, tkrs, offStart,"d:\\pklWeeklyUpdate\\")
    #rschLib.analyzeTradeTags(tradesUsed, result['rBad10'], result['idxBad10'], '亏损前10%交易',strategy_name, dtes, tkrs, offStart,"d:\\pklWeeklyUpdate\\")
    #rschLib.analyzeTradeTags(tradesUsed, result['rBad30'], result['idxBad30'], '亏损前30%交易',strategy_name, dtes, tkrs, offStart,"d:\\pklWeeklyUpdate\\")
    #rschLib.analyzeTradeTags(tradesUsed, result['rDieting'], result['idxDieting'], '第一天跌停',strategy_name, dtes, tkrs, offStart,"d:\\pklWeeklyUpdate\\")
    #get tag names
    tnames, tagNamesEn,t2 = rschLib.getTagNames()
    logTime('analyzeTradeTags')
    idxOverLapTagList=rschLib.analyzeTradeTags(tradesUsed, r, list(range(len(tradesUsed))), '所有交易',strategy_name, dtes, tkrs, offStart, "d:\\pklWeeklyUpdate\\")
    logTime('analyzeTradeTags done')
    #draw pnl and tag pnl
    importlib.reload(rschLib)
    logTime('getPnlFast')
    [dtesByTrade, pnlByTrade] = rschLib.getPnlFast(r, idxTradable, dtes, tkrs, name, tradesUsed, inTime, otTime, dayOff, timeAsFloat, toDatabase='yes', strategy_name=strategy_name)
    logTime('getPnlFast done')
    logTime('aggregatePnlAndDtes')
    [dtesPnlAggr,pnlAggr, numTrades] = rschLib.aggregatePnlAndDtes(dtesByTrade, pnlByTrade)
    logTime('aggregatePnlAndDtes done')
    rschLib.drawPNL(dtesPnlAggr, pnlAggr, dtes, strategy_name,showFigure='no', toDatabase='yes')
    for i in range(len(tnames)):
        tagName = tnames[i]
        [dtesWithTag, pnlWithTag,n] = rschLib.aggregatePnlAndDtes(dtesByTrade[idxOverLapTagList[i]],pnlByTrade[idxOverLapTagList[i]])
        rschLib.drawPNL(dtesWithTag, pnlWithTag, dtes, strategy_name, showFigure='no', toDatabase='yes', dateStart=dtesPnlAggr[0], pnlType=tagName)
        rschLib.drawPNL(dtesWithTag, pnlWithTag, dtes, strategy_name+'+'+tagNamesEn[i], showFigure='no',  toDatabase='yes', dateStart=dtesPnlAggr[0], pnlType='pnl')
        # 求非涨停，以及伴随有tagName标签的交易，绘制价格变化曲线
        idxTradableAndHasTag = list(np.intersect1d(np.nonzero(idxTradable)[0], np.array(idxOverLapTagList[i])))
        rschLib.drawPriceChange(r[idxTradableAndHasTag,:], strategy_name, timeLabels=timeLabels, title=tagName,  tp=tradeArea, showGraph='no')
    # control group and optimal group
    [rawInTime, rawOtTime] = rschLib.getDefaultTradeTime(offStart)
    idxTradableRaw = np.isfinite(r_withnan[:, rawInTime])
    [dtesByTradeRaw, pnlByTradeRaw] = rschLib.getPnlFast(r, idxTradableRaw, dtes, tkrs, name, tradesUsed, rawInTime, rawOtTime, dayOff, timeAsFloat, toDatabase='yes', strategy_name=strategy_name)
    [dtesPnlAggrRaw, pnlAggrRaw, numTradesRaw] = rschLib.aggregatePnlAndDtes(dtesByTradeRaw, pnlByTradeRaw)
    rschLib.drawPNL(dtesPnlAggrRaw, pnlAggrRaw, dtes, strategy_name, showFigure='yes', toDatabase='yes', pnlType='rawPnl')
    [bestInTime, bestOtTime] = rschLib.getOptimalTradeTime(r[idxTradable, :], rawInTime)
    idxTradableBestInTime = np.isfinite(r_withnan[:, bestInTime])
    [dtesByTradeBest, pnlByTradeBest] = rschLib.getPnlFast(r, idxTradableBestInTime, dtes, tkrs, name, tradesUsed, bestInTime, bestOtTime, dayOff, timeAsFloat, toDatabase='no', strategy_name=strategy_name)
    [dtesPnlAggrBest, pnlAggrBest, k] = rschLib.aggregatePnlAndDtes(dtesByTradeBest, pnlByTradeBest)
    rschLib.drawPNL(dtesPnlAggrBest, pnlAggrBest, dtes, strategy_name, showFigure='yes', toDatabase='yes', pnlType='optimalTradeTimePnl')
    rschLib.updateStrategyBackTest(strategy_name, 'bestInTimePoint', int(bestInTime))
    rschLib.updateStrategyBackTest(strategy_name, 'bestOtTimePoint', int(bestOtTime))
    rschLib.updateStrategyBackTest(strategy_name, 'bestInTimeLabel', timeLabels[bestInTime])
    rschLib.updateStrategyBackTest(strategy_name, 'bestOtTimeLabel', timeLabels[bestOtTime])
    rschLib.updateStrategyBackTest(strategy_name, 'rawInTimePoint', int(rawInTime))
    rschLib.updateStrategyBackTest(strategy_name, 'rawOtTimePoint', int(rawOtTime))
    rschLib.updateStrategyBackTest(strategy_name, 'rawInTimeLabel', timeLabels[rawInTime])
    rschLib.updateStrategyBackTest(strategy_name, 'rawOtTimeLabel', timeLabels[rawOtTime])
    #analysis of number of trades vs performance
    importlib.reload(rschLib)
    rschLib.pnlVsNumtrades(pnlAggr, numTrades, strategy_name, toDatabase='yes')
    rschLib.saveOffStart(strategy_name, offStart)
    

In [4]:
importlib.reload(rschLib)
strategy_names = [x['file'] for x in rschLib.tagDict().values()]
offStarts = [x['off_start'] for x in rschLib.tagDict().values()]
tagNames = list(rschLib.tagDict().keys())
maxD = 5


In [ ]:
%load_ext line_profiler
importlib.reload(rschLib)
for (i,strategy_name) in enumerate(strategy_names):
    if i<51:
        continue
    print(i)
    importlib.reload(rschLib)
    offStart = offStarts[i]
    [inTime, otTime] = rschLib.getDefaultTradeTime(offStart)
    generateTagAnalysis(tagNames[i])
    analyzeStrategy(strategy_name, offStart, dtes, name, tkrs, maxD, inTime, otTime)


51
5日乖离率正常


C:\ProgramData\Anaconda3\envs\research\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  
C:\ProgramData\Anaconda3\envs\research\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\envs\research\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\envs\research\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


getTradesFast 2020-12-14 17:17:10.733611


In [31]:
# importlib.reload(rschLib)
# %load_ext line_profiler
# %lprun -f generateTagAnalysis generateTagAnalysis(tagName)